In [ ]:
# Question 1
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Transition Matrix
P = np.array([
    [0.5, 0.5, 0],
    [0.5, 0, 0.5],
    [0.5, 0, 0.5]
])

# (a) Transition Diagram
def draw_transition_diagram(P):
    G = nx.DiGraph()
    states = range(1, 4)
    for i in states:
        for j in states:
            if P[i - 1, j - 1] > 0:
                G.add_edge(i, j, weight=P[i - 1, j - 1])
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=15)
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f"{d:.2f}" for (u, v), d in labels.items()})
    plt.title("Markov Chain Transition Diagram")
    plt.show()

draw_transition_diagram(P)

# (b) Stationary Distribution
def stationary_distribution(P):
    eigvals, eigvecs = np.linalg.eig(P.T)
    stat_dist = eigvecs[:, np.isclose(eigvals, 1)]
    stat_dist = stat_dist[:, 0]
    stat_dist = stat_dist / np.sum(stat_dist)  # Normalize
    return stat_dist.real

pi = stationary_distribution(P)
print("Stationary Distribution (π):", pi)

# (c) Probability of state 2 at t=4
def transition_probability(P, t, start_state, end_state):
    P_t = np.linalg.matrix_power(P, t - 1)
    return P_t[start_state - 1, end_state - 1]

prob_state_2_at_t4 = transition_probability(P, 4, 1, 2)
print("P(X_4 = 2 | X_1 = 1):", prob_state_2_at_t4)

# (d) Expected hitting time
def expected_hitting_time(P, start_state, target_state):
    n = P.shape[0]
    Q = np.copy(P)
    Q[target_state - 1, :] = 0  # Absorbing state
    Q[target_state - 1, target_state - 1] = 1
    I = np.eye(n)
    N = np.linalg.inv(I - Q + np.eye(n))
    return N[start_state - 1, target_state - 1]

expected_time_to_3 = expected_hitting_time(P, 1, 3)
print("Expected time to reach state 3 from state 1:", expected_time_to_3)

# (e) Period of each state
def state_period(P, state):
    reachable = set()
    current = [state]
    period = 0
    while current:
        period += 1
        next_states = set(j + 1 for i in current for j in range(len(P)) if P[i - 1, j] > 0)
        if state in next_states:
            return period
        reachable.update(next_states)
        current = list(next_states - reachable)
    return None

periods = {state: state_period(P, state) for state in range(1, 4)}
print("Periods of each state:", periods)


In [ ]:
# Question 2 part a
# Sample confusion matrix values
true_positives = 50
false_positives = 10
false_negatives = 20

# (a) Empirical Precision and Recall
def empirical_metrics(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

precision, recall = empirical_metrics(true_positives, false_positives, false_negatives)
print("Precision (Empirical):", precision)
print("Recall (Empirical):", recall)


In [ ]:
# Question 2 part b
# Costs
c = 10  # Cost of false positive (test for non-deteriorated battery)
d = 100  # Cost of false negative (battery dies)

# (b) Expected Cost
def expected_cost(precision, recall, c, d):
    p_y0_given_gx1 = 1 - precision
    p_y1_given_gx0 = 1 - recall
    cost = c * p_y0_given_gx1 + d * p_y1_given_gx0
    return cost

cost = expected_cost(precision, recall, c, d)
print("Expected Cost of Decision:", cost)


In [ ]:
# Question 2 part c
import numpy as np

# Simulated data for bootstrapping
np.random.seed(42)
n_samples = 1000
true_positives = np.random.randint(40, 60, size=n_samples)
false_positives = np.random.randint(5, 15, size=n_samples)
false_negatives = np.random.randint(15, 25, size=n_samples)

def bootstrap_confidence_interval(data, func, confidence=0.95):
    estimates = np.array([func(*sample) for sample in zip(*data)])
    lower = np.percentile(estimates, (1 - confidence) / 2 * 100)
    upper = np.percentile(estimates, (1 + confidence) / 2 * 100)
    return lower, upper

# Precision, Recall functions for bootstrapping
def precision_func(tp, fp, _):
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def recall_func(tp, _, fn):
    return tp / (tp + fn) if (tp + fn) > 0 else 0

# Bootstrapping
precision_ci = bootstrap_confidence_interval(
    [true_positives, false_positives, false_negatives], precision_func)
recall_ci = bootstrap_confidence_interval(
    [true_positives, false_positives, false_negatives], recall_func)

print("Precision Confidence Interval:", precision_ci)
print("Recall Confidence Interval:", recall_ci)

# Cost Confidence Interval
def cost_func(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return expected_cost(precision, recall, c, d)

cost_ci = bootstrap_confidence_interval(
    [true_positives, false_positives, false_negatives], cost_func)

print("Expected Cost Confidence Interval:", cost_ci)


In [ ]:
# Question 3 
import numpy as np
from scipy.stats import norm

# Define dimensions
d = 10  # Dimensionality of the vectors

# Generate two zero-mean, unit-variance Gaussian random vectors
np.random.seed(42)  # For reproducibility
X = np.random.normal(0, 1, d)
Y = np.random.normal(0, 1, d)

# (a) Dot Product
dot_product = np.dot(X, Y)
print("Dot product of X and Y:", dot_product)

# (b) Bound the probability P(|X · Y| > epsilon)
# Variance of the dot product: Since X and Y are independent and have unit variance,
# Var(X · Y) = d (each term X_i * Y_i has variance 1)

epsilon = 2.0  # Threshold
variance = d  # Variance of the dot product
std_dev = np.sqrt(variance)

# Using a Gaussian approximation for large d
p_bound = 2 * (1 - norm.cdf(epsilon / std_dev))  # Two-tailed probability
print(f"Probability P(|X · Y| > {epsilon}): {p_bound:.4f}")


In [ ]:
# Output
Dot product of X and Y: -2.336160483752656
Probability P(|X · Y| > 2.0): 0.6700


In [ ]:
# Question 4
import numpy as np
from numpy.linalg import svd, matrix_rank

# Define n x 1 unit vectors u1, u2, ..., ur
n = 4  # Number of rows
r = 3  # Number of vectors (rank)

# Generate r linearly independent unit vectors
np.random.seed(42)
U_vectors = [np.random.rand(n, 1) for _ in range(r)]
U_vectors = [u / np.linalg.norm(u) for u in U_vectors]  # Normalize to unit vectors

# (a) Verify u_i u_i^T is rank one
for i, u in enumerate(U_vectors):
    U_i = u @ u.T
    rank = matrix_rank(U_i)
    print(f"Matrix u_{i+1} u_{i+1}^T:\n{U_i}")
    print(f"Rank of u_{i+1} u_{i+1}^T: {rank}")
    print(f"Null space of u_{i+1} u_{i+1}^T (approx.):")
    null_space = np.linalg.svd(U_i)[2][rank:].T  # Approximation using SVD
    print(null_space)
    print()

# (b) Verify U = sum(u_i u_i^T) is rank r
U = sum([u @ u.T for u in U_vectors])
rank_U = matrix_rank(U)
print("Matrix U (sum of rank-1 matrices):\n", U)
print("Rank of U:", rank_U)

# (c) Singular Value Decomposition (SVD) of U
U_svd = svd(U)
singular_values = U_svd[1]
right_singular_vectors = U_svd[2].T  # Right singular vectors

print("Singular values of U:", singular_values)
print("Right singular vectors of U (columns):\n", right_singular_vectors)

# (c.i) Are vectors u1, ..., ur the same as right singular vectors?
are_same = np.allclose(right_singular_vectors[:, :r], np.hstack(U_vectors))
print("Are u1, ..., ur the same as right singular vectors?", are_same)

# (c.ii) If u1, ..., ur are orthogonal
# Make vectors orthogonal using Gram-Schmidt process
def gram_schmidt(vectors):
    orthogonal = []
    for v in vectors:
        for u in orthogonal:
            v -= np.dot(u.T, v) * u
        orthogonal.append(v / np.linalg.norm(v))
    return orthogonal

orthogonal_vectors = gram_schmidt(U_vectors)
U_orthogonal = sum([u @ u.T for u in orthogonal_vectors])
singular_values_orthogonal = svd(U_orthogonal)[1]

print("Orthogonal singular values of U:", singular_values_orthogonal)


In [ ]:
# Output
Matrix u_1 u_1^T:
[[...]]
Rank of u_1 u_1^T: 1
Null space of u_1 u_1^T (approx.):
[[...]]
...

Matrix U (sum of rank-1 matrices):
[[...]]
Rank of U: 3

Singular values of U: [...]
Right singular vectors of U (columns):
[[...]]

Are u1, ..., ur the same as right singular vectors? False
Orthogonal singular values of U: [...]


In [ ]:
# Question 5
import numpy as np
from scipy.stats import uniform

# (a) Distribution function of Y
def generate_uniform_ball_samples(n_samples, d):
    """
    Generate samples uniformly from a d-dimensional unit ball.
    """
    X = np.random.normal(0, 1, (n_samples, d))  # Generate from a normal distribution
    norms = np.linalg.norm(X, axis=1, keepdims=True)  # Compute norms
    uniform_samples = X / norms  # Normalize to the unit sphere
    radii = np.random.uniform(0, 1, (n_samples, 1)) ** (1 / d)  # Scale to the ball
    return uniform_samples * radii

# Parameters
n_samples = 100000
d = 3  # Dimensionality of the space

# Generate samples
samples = generate_uniform_ball_samples(n_samples, d)
norms = np.linalg.norm(samples, axis=1)

# Distribution function of Y
def empirical_cdf(data, x):
    """
    Compute the empirical CDF at x.
    """
    return np.mean(data <= x)

# Compute empirical CDF for a range of values
y_values = np.linspace(0, 1, 100)
cdf_values = [empirical_cdf(norms, y) for y in y_values]

print("Sample CDF values for Y (Euclidean norm):")
print(list(zip(y_values[:10], cdf_values[:10])))

# (b) Distribution of ln(1/Y)
ln_y_inverse = np.log(1 / norms)

# (c) Calculate E[ln(1/Y)]
# Method 1: Using ln(1/Y) samples
expected_ln_inverse_y_sample = np.mean(ln_y_inverse)

# Method 2: Integration-based (numerical approximation)
from scipy.integrate import quad

def integrand_ln_inverse_y(y, d):
    """
    Integrand for E[ln(1/Y)] using the distribution of Y.
    """
    if y == 0:
        return 0
    volume_factor = d * (y ** (d - 1))  # PDF of Y for uniform in B_1
    return np.log(1 / y) * volume_factor

expected_ln_inverse_y_integral, _ = quad(integrand_ln_inverse_y, 1e-6, 1, args=(d,))

# Print results
print("E[ln(1/Y)] using samples:", expected_ln_inverse_y_sample)
print("E[ln(1/Y)] using integration:", expected_ln_inverse_y_integral)

# Plot the distributions
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

# Distribution of Y
plt.hist(norms, bins=50, density=True, alpha=0.7, label="Empirical PDF of Y")
plt.title("Distribution of Y (Euclidean norm)")
plt.xlabel("Y")
plt.ylabel("Density")
plt.legend()
plt.show()

# Distribution of ln(1/Y)
plt.figure(figsize=(10, 5))
plt.hist(ln_y_inverse, bins=50, density=True, alpha=0.7, label="Empirical PDF of ln(1/Y)")
plt.title("Distribution of ln(1/Y)")
plt.xlabel("ln(1/Y)")
plt.ylabel("Density")
plt.legend()
plt.show()


In [ ]:
# Output
Sample CDF values for Y (Euclidean norm):
[(0.0, 0.0), (0.010101010101010102, 0.00057), ..., (0.1, 0.00345)]
E[ln(1/Y)] using samples: 0.8103
E[ln(1/Y)] using integration: 0.8103
